# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234684668513                   -0.50    7.0
  2   -7.250200667105       -1.81       -1.39    1.0
  3   -7.251029933321       -3.08       -1.87    4.0
  4   -7.250872218103   +   -3.80       -1.83    3.0
  5   -7.251336284512       -3.33       -2.92    2.0
  6   -7.251338430973       -5.67       -3.31    2.0
  7   -7.251338792102       -6.44       -4.15    3.0
  8   -7.251338797313       -8.28       -4.48    4.0
  9   -7.251338798563       -8.90       -4.96    3.0
 10   -7.251338798695       -9.88       -5.52    2.0
 11   -7.251338798704      -11.05       -6.06    3.0
 12   -7.251338798704      -12.15       -6.40    3.0
 13   -7.251338798705      -13.03       -6.78    1.0
 14   -7.251338798705      -13.51       -7.48    3.0
 15   -7.251338798705      -15.05       -7.87    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07317096850756724
[ Info: Arnoldi iteration step 2: normres = 0.3122015128057041
[ Info: Arnoldi iteration step 3: normres = 0.9070162277975954
[ Info: Arnoldi iteration step 4: normres = 0.4209388024148459
[ Info: Arnoldi iteration step 5: normres = 0.6930774148624578
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.53e-01, 9.61e-02, 6.27e-01, 2.32e-01, 2.91e-02)
[ Info: Arnoldi iteration step 6: normres = 0.1880822410426338
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.21e-02, 9.60e-02, 1.03e-01, 9.05e-02, 3.36e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09387224116158094
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.01e-03, 8.34e-03, 8.51e-03, 7.25e-02, 5.21e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13209068558373677
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.80e-0